## 1. Import packages and Load dataset

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
pip install --index-url https://test.pypi.org/simple/ --no-deps my_krml_ratana

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
from my_krml_ratana.data.sets import df_report

In [10]:
df = pd.read_csv('../../data/interim/df_master.csv')

In [11]:
df

,cc_num,gender,city,state,zip,lat,long,city_pop,acct_num,job_category,age,trans_num,category,amt,is_fraud,merchant,merch_lat,merch_long,timestamp
0,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,65,91ab12e73ef38206e1121e9648d2408d,gas_transport,69.12,0,Phillips Group,39.491416,-75.588522,2019-05-24 17:39:10
1,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,65,071553d533a6822a4431c354c434ddcb,grocery_pos,68.11,0,Tucker Ltd,40.890319,-75.573359,2019-09-25 15:31:59
2,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,65,0cfad38ef15e4749eff68dc83f62c151,misc_net,40.35,0,Dixon PLC,39.244958,-74.475327,2019-12-24 16:40:01
3,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,65,5782693d7c70f062f258cb30bfa8900f,grocery_pos,96.22,0,Lambert-Cooper,39.656925,-75.802342,2019-10-18 19:50:38
4,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,65,35fd7db657d7e30dd608c37f7798186e,gas_transport,71.89,0,Griffith LLC,40.313342,-74.220434,2019-02-10 23:13:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4260899,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,82,4eafc5b16da625c3436f2484d8dcabcb,health_fitness,7.47,0,"Horton, Ware and Reynolds",27.707908,-82.320308,2021-07-30 09:22:23
4260900,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,82,041b18f68b893b68e7aa29c16a08a08d,food_dining,2.30,0,Hanson and Sons,27.896372,-83.387356,2021-06-28 11:38:46
4260901,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,82,a26df5348dfee4839f0fa00155e2ed1c,health_fitness,5.82,0,White LLC,27.110602,-82.426821,2021-08-24 09:37:49
4260902,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,82,e388fc8055b1426c62640baaae10108f,health_fitness,9.16,0,Snyder-Johnson,28.110510,-81.891797,2021-08-26 11:16:35


## 2. Creating target variable

Since our goal is to predict monthly spending of each customer for next month we need to create a monthly spending amount (target variable)

Before building the model we need to consider if there are enough data points to create one model per customers. This is important when we decide train/test/validation split
   

In [13]:
transaction_counts = df.groupby('cc_num').size().reset_index(name='transaction_count')

In [14]:
transaction_counts

,cc_num,transaction_count
0,60400268763,5101
1,60406155816,6910
2,60429276409,7631
3,60432200743,5454
4,60434956110,9476
...,...,...
978,4966177182498980888,3639
979,4967150066664110652,7630
980,4968827697040953141,4384
981,4983665528993160877,5096


In [17]:
# Create year and month features from timestamp

df['timestamp'] = pd.to_datetime(df['timestamp'])

df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month

In [18]:
df

,cc_num,gender,city,state,zip,lat,long,city_pop,acct_num,job_category,...,trans_num,category,amt,is_fraud,merchant,merch_lat,merch_long,timestamp,year,month
0,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,91ab12e73ef38206e1121e9648d2408d,gas_transport,69.12,0,Phillips Group,39.491416,-75.588522,2019-05-24 17:39:10,2019,5
1,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,071553d533a6822a4431c354c434ddcb,grocery_pos,68.11,0,Tucker Ltd,40.890319,-75.573359,2019-09-25 15:31:59,2019,9
2,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,0cfad38ef15e4749eff68dc83f62c151,misc_net,40.35,0,Dixon PLC,39.244958,-74.475327,2019-12-24 16:40:01,2019,12
3,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,5782693d7c70f062f258cb30bfa8900f,grocery_pos,96.22,0,Lambert-Cooper,39.656925,-75.802342,2019-10-18 19:50:38,2019,10
4,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,35fd7db657d7e30dd608c37f7798186e,gas_transport,71.89,0,Griffith LLC,40.313342,-74.220434,2019-02-10 23:13:20,2019,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4260899,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,4eafc5b16da625c3436f2484d8dcabcb,health_fitness,7.47,0,"Horton, Ware and Reynolds",27.707908,-82.320308,2021-07-30 09:22:23,2021,7
4260900,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,041b18f68b893b68e7aa29c16a08a08d,food_dining,2.30,0,Hanson and Sons,27.896372,-83.387356,2021-06-28 11:38:46,2021,6
4260901,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,a26df5348dfee4839f0fa00155e2ed1c,health_fitness,5.82,0,White LLC,27.110602,-82.426821,2021-08-24 09:37:49,2021,8
4260902,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,e388fc8055b1426c62640baaae10108f,health_fitness,9.16,0,Snyder-Johnson,28.110510,-81.891797,2021-08-26 11:16:35,2021,8


In [25]:
# Number of transaction per year
transaction_counts = df.groupby(['cc_num', 'year']).size().reset_index(name='transaction_count')

In [26]:
transaction_counts.head(50)

,cc_num,year,transaction_count
0,60400268763,2018,1
1,60400268763,2019,1130
2,60400268763,2020,1110
3,60400268763,2021,1092
4,60400268763,2022,1768
5,60406155816,2018,1
6,60406155816,2019,1446
7,60406155816,2020,1436
8,60406155816,2021,1442
9,60406155816,2022,2585


In [27]:
# Number of transaction per month
transaction_counts = df.groupby(['cc_num', 'month']).size().reset_index(name='transaction_count')

In [30]:
transaction_counts.describe()

,cc_num,month,transaction_count
count,1.167900e+04,11679.000000,11679.000000
mean,3.718194e+17,6.497474,364.834661
std,1.236831e+18,3.452162,239.906251
min,6.040027e+10,1.000000,3.000000
25%,1.800364e+14,3.000000,189.000000
50%,3.517536e+15,6.000000,323.000000
75%,4.536938e+15,9.000000,490.000000
max,4.986227e+18,12.000000,1748.000000


The minimum transaction per month for each customers is 3. 

### [2.1] Aggregate monthly spending for each customers

In [34]:
df.columns

Index(['cc_num', 'gender', 'city', 'state', 'zip', 'lat', 'long', 'city_pop',
       'acct_num', 'job_category', 'age', 'trans_num', 'category', 'amt',
       'is_fraud', 'merchant', 'merch_lat', 'merch_long', 'timestamp', 'year',
       'month'],
      dtype='object')

In [36]:
# Total monthly spending
monthly_spending = df.groupby(['cc_num', 'year', 'month'])['amt'].sum().reset_index()
monthly_spending

,cc_num,year,month,amt
0,60400268763,2018,12,5.39
1,60400268763,2019,1,3152.91
2,60400268763,2019,2,9242.60
3,60400268763,2019,3,5701.32
4,60400268763,2019,4,6016.01
...,...,...,...,...
43797,4986226992066617996,2022,8,4962.25
43798,4986226992066617996,2022,9,4168.69
43799,4986226992066617996,2022,10,12604.28
43800,4986226992066617996,2022,11,3604.19


In [37]:
monthly_spending.rename(columns={'amt': 'total_monthly_spending'}, inplace=True)

In [38]:
monthly_spending

,cc_num,year,month,total_monthly_spending
0,60400268763,2018,12,5.39
1,60400268763,2019,1,3152.91
2,60400268763,2019,2,9242.60
3,60400268763,2019,3,5701.32
4,60400268763,2019,4,6016.01
...,...,...,...,...
43797,4986226992066617996,2022,8,4962.25
43798,4986226992066617996,2022,9,4168.69
43799,4986226992066617996,2022,10,12604.28
43800,4986226992066617996,2022,11,3604.19


## 3. Feature Engineering

### [3.1] Monthly spending by Categories

In [40]:
# Monthly spending by category
# Use pivot table to calculate spending by category
category_spending = df.pivot_table(
    index = ['cc_num', 'year', 'month'],
    columns = 'category',
    values = 'amt',
    aggfunc = 'sum',
    fill_value = 0
).reset_index()


In [41]:
category_spending

category,cc_num,year,month,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,60400268763,2018,12,0.00,0.00,5.39,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
1,60400268763,2019,1,0.00,0.00,3152.91,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
2,60400268763,2019,2,0.00,0.00,9242.60,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
3,60400268763,2019,3,0.00,0.00,5701.32,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
4,60400268763,2019,4,0.00,0.00,6016.01,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43797,4986226992066617996,2022,8,4452.26,340.38,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,169.61,0.00
43798,4986226992066617996,2022,9,0.00,4168.69,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
43799,4986226992066617996,2022,10,0.00,0.00,0.00,0.0,0.0,11452.41,1151.87,0.00,0.0,0.0,0.00,0.0,0.00,0.00
43800,4986226992066617996,2022,11,0.00,0.00,0.00,0.0,0.0,0.00,1978.56,1625.63,0.0,0.0,0.00,0.0,0.00,0.00


In [42]:
# Lag for total monthly spending
monthly_spending['spending_lag_1'] = monthly_spending.groupby('cc_num')['total_monthly_spending'].shift(1)


In [44]:
# 2nd order lag
monthly_spending['spending_lag_2'] = monthly_spending.groupby('cc_num')['total_monthly_spending'].shift(2)

In [45]:
# 3rd order lag
monthly_spending['spending_lag_3'] = monthly_spending.groupby('cc_num')['total_monthly_spending'].shift(3)

In [50]:
monthly_spending

,cc_num,year,month,total_monthly_spending,spending_lag_1,spending_lag_2,spending_lag_3
0,60400268763,2018,12,5.39,NaN,NaN,NaN
1,60400268763,2019,1,3152.91,5.39,NaN,NaN
2,60400268763,2019,2,9242.60,3152.91,5.39,NaN
3,60400268763,2019,3,5701.32,9242.60,3152.91,5.39
4,60400268763,2019,4,6016.01,5701.32,9242.60,3152.91
...,...,...,...,...,...,...,...
43797,4986226992066617996,2022,8,4962.25,6822.38,4743.58,4824.19
43798,4986226992066617996,2022,9,4168.69,4962.25,6822.38,4743.58
43799,4986226992066617996,2022,10,12604.28,4168.69,4962.25,6822.38
43800,4986226992066617996,2022,11,3604.19,12604.28,4168.69,4962.25


In [48]:
# Number of transactions per month
monthly_trans = df.groupby(['cc_num', 'year', 'month'])['trans_num'].count().reset_index()
monthly_trans.rename(columns={'trans_num': 'num_transactions'}, inplace = True)
monthly_trans

,cc_num,year,month,num_transactions
0,60400268763,2018,12,1
1,60400268763,2019,1,64
2,60400268763,2019,2,58
3,60400268763,2019,3,92
4,60400268763,2019,4,72
...,...,...,...,...
43797,4986226992066617996,2022,8,127
43798,4986226992066617996,2022,9,105
43799,4986226992066617996,2022,10,110
43800,4986226992066617996,2022,11,102


In [53]:
# Merge data
final_data = pd.merge(monthly_spending, category_spending, on=['cc_num', 'year', 'month'])
final_data = pd.merge(final_data, monthly_trans, on=['cc_num', 'year', 'month'])


In [55]:
final_data.head(20)

,cc_num,year,month,total_monthly_spending,spending_lag_1,spending_lag_2,spending_lag_3,entertainment,food_dining,gas_transport,...,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel,num_transactions
0,60400268763,2018,12,5.39,NaN,NaN,NaN,0.0,0.0,5.39,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,1
1,60400268763,2019,1,3152.91,5.39,NaN,NaN,0.0,0.0,3152.91,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,64
2,60400268763,2019,2,9242.60,3152.91,5.39,NaN,0.0,0.0,9242.60,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,58
3,60400268763,2019,3,5701.32,9242.60,3152.91,5.39,0.0,0.0,5701.32,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,92
4,60400268763,2019,4,6016.01,5701.32,9242.60,3152.91,0.0,0.0,6016.01,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,72
5,60400268763,2019,5,6040.05,6016.01,5701.32,9242.60,0.0,0.0,6040.05,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,93
6,60400268763,2019,6,8893.32,6040.05,6016.01,5701.32,0.0,0.0,5029.69,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,103
7,60400268763,2019,7,6339.15,8893.32,6040.05,6016.01,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,120
8,60400268763,2019,8,6750.15,6339.15,8893.32,6040.05,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,109
9,60400268763,2019,9,7976.41,6750.15,6339.15,8893.32,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,95


In [60]:
df

,cc_num,gender,city,state,zip,lat,long,city_pop,acct_num,job_category,...,trans_num,category,amt,is_fraud,merchant,merch_lat,merch_long,timestamp,year,month
0,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,91ab12e73ef38206e1121e9648d2408d,gas_transport,69.12,0,Phillips Group,39.491416,-75.588522,2019-05-24 17:39:10,2019,5
1,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,071553d533a6822a4431c354c434ddcb,grocery_pos,68.11,0,Tucker Ltd,40.890319,-75.573359,2019-09-25 15:31:59,2019,9
2,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,0cfad38ef15e4749eff68dc83f62c151,misc_net,40.35,0,Dixon PLC,39.244958,-74.475327,2019-12-24 16:40:01,2019,12
3,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,5782693d7c70f062f258cb30bfa8900f,grocery_pos,96.22,0,Lambert-Cooper,39.656925,-75.802342,2019-10-18 19:50:38,2019,10
4,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,35fd7db657d7e30dd608c37f7798186e,gas_transport,71.89,0,Griffith LLC,40.313342,-74.220434,2019-02-10 23:13:20,2019,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4260899,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,4eafc5b16da625c3436f2484d8dcabcb,health_fitness,7.47,0,"Horton, Ware and Reynolds",27.707908,-82.320308,2021-07-30 09:22:23,2021,7
4260900,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,041b18f68b893b68e7aa29c16a08a08d,food_dining,2.30,0,Hanson and Sons,27.896372,-83.387356,2021-06-28 11:38:46,2021,6
4260901,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,a26df5348dfee4839f0fa00155e2ed1c,health_fitness,5.82,0,White LLC,27.110602,-82.426821,2021-08-24 09:37:49,2021,8
4260902,343251790447085,M,Saint Petersburg,FL,33710,27.7898,-82.7243,341043,298026601683,Engineering,...,e388fc8055b1426c62640baaae10108f,health_fitness,9.16,0,Snyder-Johnson,28.110510,-81.891797,2021-08-26 11:16:35,2021,8


In [62]:
df[df['cc_num'] == 4218196001337]

,cc_num,gender,city,state,zip,lat,long,city_pop,acct_num,job_category,...,trans_num,category,amt,is_fraud,merchant,merch_lat,merch_long,timestamp,year,month
0,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,91ab12e73ef38206e1121e9648d2408d,gas_transport,69.12,0,Phillips Group,39.491416,-75.588522,2019-05-24 17:39:10,2019,5
1,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,071553d533a6822a4431c354c434ddcb,grocery_pos,68.11,0,Tucker Ltd,40.890319,-75.573359,2019-09-25 15:31:59,2019,9
2,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,0cfad38ef15e4749eff68dc83f62c151,misc_net,40.35,0,Dixon PLC,39.244958,-74.475327,2019-12-24 16:40:01,2019,12
3,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,5782693d7c70f062f258cb30bfa8900f,grocery_pos,96.22,0,Lambert-Cooper,39.656925,-75.802342,2019-10-18 19:50:38,2019,10
4,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,35fd7db657d7e30dd608c37f7798186e,gas_transport,71.89,0,Griffith LLC,40.313342,-74.220434,2019-02-10 23:13:20,2019,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3273,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,d7b20af8b8855bed2aafcdade1b2ecba,home,9.38,0,"Baker, Jackson and Jackson",40.172804,-74.367905,2022-10-02 04:21:27,2022,10
3274,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,4bbe653774d684f87e352f3af2d3dfc1,home,7.96,0,"Pratt, Cox and Brown",39.821362,-75.793384,2022-10-31 10:57:10,2022,10
3275,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,a2de568f42a1f7cedb9d7dc84a14812b,personal_care,9.62,0,Lewis-Castillo,39.547240,-75.740526,2022-12-23 04:24:28,2022,12
3276,4218196001337,M,Ambler,PA,19002,40.1809,-75.2156,32412,888022315787,Finance,...,901831a40b3c645a8822e04af88f35a9,personal_care,8.69,0,Santiago-Sheppard,40.495550,-76.146362,2022-12-22 12:57:01,2022,12
